In [2]:
%load_ext nb_black

import os
import sys

sys.path.append(
    os.path.abspath("/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/codebase")
)
import torch.backends.cudnn as cudnn
import random
import time
from collections import OrderedDict
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.functional import one_hot
import sklearn.metrics as metrics
import utils
from Explainer.loss_F import loss_fn_kd, entropy_loss
from Explainer.models.Gated_Logic_Net import Gated_Logic_Net
from Explainer.models.explainer import Explainer
from Explainer.models.pi import Pi
from dataset.dataset_cubs import Dataset_cub_for_explainer
from dataset.utils_dataset import get_dataset_with_image_and_attributes
from Explainer.loss_F import loss_fn_kd, entropy_loss, Selective_Distillation_Loss
from Explainer.models.concepts import Conceptizator

import matplotlib.pyplot as plt

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [30]:
device = utils.get_device()
print(f"Device: {device}")

Device: cuda


<IPython.core.display.Javascript object>

In [5]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "iter1",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[0])
print(args.cov[0])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter1"
expert_type = "explainer/accuracy"
output = "g_outputs"


# val
val_tensor_preds_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "val_tensor_preds.pt"
    )
)

val_tensor_preds_bb_1 = torch.load(
    os.path.join(
        root, experiment, iteration, expert_type, output, "val_tensor_preds_bb.pt"
    )
)

val_tensor_y_1 = torch.load(
    os.path.join(root, experiment, iteration, expert_type, output, "val_tensor_y.pt")
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_1.size())
print(val_tensor_preds_bb_1.size())
print(val_tensor_y_1.size())

val_auroc_g, _ = utils.compute_AUC(gt=val_tensor_y_1, pred=val_tensor_preds_1[:, 1])
val_auroc_bb, _ = utils.compute_AUC(gt=val_tensor_y_1, pred=val_tensor_preds_bb_1[:, 1])
print(f"Auroc of Residual: {val_auroc_g} || Auroc of BB: {val_auroc_bb}")
print(f"Proportional Auroc: {val_auroc_g * (val_tensor_preds_bb_1.size(0)/2003)}")



########################
0.01
0.2


 << Val sizes >>
torch.Size([799, 2])
torch.Size([799, 2])
torch.Size([799])
Auroc of Residual: 0.5 || Auroc of BB: 0.5
Proportional Auroc: 0.19945082376435347


<IPython.core.display.Javascript object>

In [9]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter2",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter2"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
val_tensor_preds_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds.pt",
    )
)

val_tensor_preds_bb_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds_bb.pt",
    )
)

val_tensor_y_2 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_2.size())
print(val_tensor_preds_bb_2.size())
print(val_tensor_y_2.size())

val_auroc_g, _ = utils.compute_AUC(gt=val_tensor_y_2, pred=val_tensor_preds_2[:, 1])
val_auroc_bb, _ = utils.compute_AUC(gt=val_tensor_y_2, pred=val_tensor_preds_bb_2[:, 1])
print(f"Auroc of Residual: {val_auroc_g} || Auroc of BB: {val_auroc_bb}")
print(f"Proportional Auroc: {val_auroc_g * (val_tensor_preds_bb_2.size(0)/2003)}")




########################
0.01
0.2


 << Val sizes >>
torch.Size([231, 2])
torch.Size([231, 2])
torch.Size([231])
Auroc of Residual: 0.6615720524017468 || Auroc of BB: 0.8165938864628821
Proportional Auroc: 0.07629712636285747


<IPython.core.display.Javascript object>

In [7]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter3",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter3"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
val_tensor_preds_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds.pt",
    )
)

val_tensor_preds_bb_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds_bb.pt",
    )
)

val_tensor_y_3 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_3.size())
print(val_tensor_preds_bb_3.size())
print(val_tensor_y_3.size())

val_auroc_g, _ = utils.compute_AUC(gt=val_tensor_y_3, pred=val_tensor_preds_3[:, 1])
val_auroc_bb, _ = utils.compute_AUC(gt=val_tensor_y_3, pred=val_tensor_preds_bb_3[:, 1])
print(f"Auroc of Residual: {val_auroc_g} || Auroc of BB: {val_auroc_bb}")
print(f"Proportional Auroc: {val_auroc_g * (val_tensor_preds_bb_3.size(0)/2003)}")



########################
0.01
0.2


 << Val sizes >>
torch.Size([303, 2])
torch.Size([303, 2])
torch.Size([303])
Auroc of Residual: 0.6323163138231631 || Auroc of BB: 0.7341220423412205
Proportional Auroc: 0.25223201934333866


<IPython.core.display.Javascript object>

In [34]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter4",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter4"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
val_tensor_preds_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds.pt",
    )
)

val_tensor_preds_bb_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds_bb.pt",
    )
)

val_tensor_y_4 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_4.size())
print(val_tensor_preds_bb_4.size())
print(val_tensor_y_4.size())

val_auroc_g, _ = utils.compute_AUC(gt=val_tensor_y_4, pred=val_tensor_preds_4[:, 1])
val_auroc_bb, _ = utils.compute_AUC(gt=val_tensor_y_4, pred=val_tensor_preds_bb_4[:, 1])
print(f"Auroc of Residual: {val_auroc_g} || Auroc of BB: {val_auroc_bb}")
print(f"Proportional Auroc: {val_auroc_g * (val_tensor_preds_bb_4.size(0)/2003)}")




########################
0.01
0.2


 << Val sizes >>
torch.Size([203, 2])
torch.Size([203, 2])
torch.Size([203])


<IPython.core.display.Javascript object>

In [44]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter5",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter5"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
val_tensor_preds_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds.pt",
    )
)

val_tensor_preds_bb_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds_bb.pt",
    )
)

val_tensor_y_5 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_5.size())
print(val_tensor_preds_bb_5.size())
print(val_tensor_y_5.size())


########################
0.01
0.2


 << Val sizes >>
torch.Size([126, 2])
torch.Size([126, 2])
torch.Size([126])


<IPython.core.display.Javascript object>

In [59]:
base_lr = 0.01
base_cov = 0.2
pickle_in = open(
    os.path.join(
        f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/explainer/lr_{base_lr}_epochs_500_temperature-lens_0.7_input-size-pi_2048_cov_{base_cov}_alpha_0.5_selection-threshold_0.5_lambda-lens_0.0001_alpha-KD_0.9_temperature-KD_10.0_hidden-layers_1",
        "cov_0.2",
        "iter6",
        "explainer/accuracy",
        "test_explainer_configs.pkl",
    ),
    "rb",
)
args = pickle.load(pickle_in)
n_classes = len(args.labels)
x_to_bool = 0.5
top_k_explanations = 50
concept_names = args.concept_names
print("########################")
print(args.lr[-1])
print(args.cov[-1])

use_concepts_as_pi_input = True
explainer_init = "none"

experiment_folder = (
    f"lr_{args.lr[0]}_epochs_{args.epochs}_temperature-lens_{args.temperature_lens}"
    f"_input-size-pi_{args.input_size_pi}"
    f"_cov_{args.cov[0]}_alpha_{args.alpha}_selection-threshold_{args.selection_threshold}"
    f"_lambda-lens_{args.lambda_lens}_alpha-KD_{args.alpha_KD}"
    f"_temperature-KD_{float(args.temperature_KD)}_hidden-layers_{len(args.hidden_nodes)}"
)

root = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k"
experiment = f"explainer/{experiment_folder}"
iteration = "iter6"
expert_type = "explainer/accuracy"
output = "g_outputs"

# val
val_tensor_preds_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds.pt",
    )
)

val_tensor_preds_bb_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_preds_bb.pt",
    )
)

val_tensor_y_6 = torch.load(
    os.path.join(
        root,
        experiment,
        f"cov_{args.cov[-1]}",
        iteration,
        expert_type,
        output,
        "val_tensor_y.pt",
    )
)


print("\n\n << Val sizes >>")
print(val_tensor_preds_6.size())
print(val_tensor_preds_bb_6.size())
print(val_tensor_y_6.size())


########################
0.01
0.2


 << Val sizes >>
torch.Size([154, 2])
torch.Size([154, 2])
torch.Size([154])


<IPython.core.display.Javascript object>

In [60]:
expert_type = "explainer"
output = "g_outputs"

preds = torch.cat(
    (
        val_tensor_preds_1, val_tensor_preds_2, val_tensor_preds_3, 
        val_tensor_preds_4, val_tensor_preds_5, val_tensor_preds_6
    ), dim=0
)
gt = torch.cat(
    (
        val_tensor_y_1, val_tensor_y_2, val_tensor_y_3 ,val_tensor_y_4,
        val_tensor_y_5, val_tensor_y_6
    ), dim=0
)
bb = torch.cat(
    (
        val_tensor_preds_bb_1.cpu(), val_tensor_preds_bb_2.cpu(), val_tensor_preds_bb_3.cpu(),
        val_tensor_preds_bb_4.cpu(), val_tensor_preds_bb_5.cpu(), val_tensor_preds_bb_6.cpu()
    ),dim=0
)


<IPython.core.display.Javascript object>

In [61]:
preds_tot_np = preds.argmax(dim=1).numpy()
gt_tot_np = gt.numpy()
bb_tot_np = bb.argmax(dim=1).numpy()
bb_tot_acc = metrics.accuracy_score(gt_tot_np, bb_tot_np)
g_tot_acc = metrics.accuracy_score(gt_tot_np, preds_tot_np)
g_tot_f1 = metrics.f1_score(gt_tot_np, preds_tot_np, average="micro")
g_tot_precision = metrics.precision_score(gt_tot_np, preds_tot_np, average="micro")
g_tot_recall = metrics.recall_score(gt_tot_np, preds_tot_np, average="micro")
bb_tot_recall = metrics.recall_score(gt_tot_np, bb_tot_np, average="micro")
bb_tot_f1 = metrics.f1_score(gt_tot_np, bb_tot_np, average="micro")
g_tot_cov = preds.size(0) / 2003

proba = torch.nn.Softmax(dim=1)(preds)[:, 1]
test_auroc, test_aurpc = utils.compute_AUC(gt, pred=proba)
proba_bb = torch.nn.Softmax(dim=1)(bb)[:, 1]
test_auroc_bb, test_aurpc_bb = utils.compute_AUC(gt, pred=proba_bb)

print(f"G Auroc: {test_auroc}")
print(f"BB Auroc: {test_auroc_bb}")
print(f"g_tot_acc: {g_tot_acc * 100} (%)")
print(f"bb_tot_acc: {bb_tot_acc * 100} (%)")
print(f"g_tot_f1: {g_tot_f1 * 100} (%)")
print(f"g_tot_precision: {g_tot_precision * 100} (%)")
print(f"g_tot_recall: {g_tot_recall * 100} (%)")
print(f"bb_tot_recall: {bb_tot_recall * 100} (%)")
print(f"bb_tot_f1: {bb_tot_f1 * 100} (%)")
print(f"total samples covered by g: {preds.size(0)} (out of {2003})")
print(f"total coverage by g: {g_tot_cov}")

# G Auroc: 0.9565059403437817
# BB Auroc: 0.9650636164475902
# g_tot_acc: 90.11560693641619 (%)
# bb_tot_acc: 89.1907514450867 (%)
# g_tot_f1: 90.11560693641619 (%)
# g_tot_precision: 90.11560693641619 (%)
# g_tot_recall: 90.11560693641619 (%)
# bb_tot_recall: 89.1907514450867 (%)
# bb_tot_f1: 89.1907514450867 (%)
# total samples covered by g: 1730 (out of 2003)
# total coverage by g: 0.8637044433349975

# G Auroc: 0.9633698589989904
# BB Auroc: 0.9714171741183367
# g_tot_acc: 95.0661853188929 (%)
# bb_tot_acc: 95.00601684717208 (%)
# g_tot_f1: 95.0661853188929 (%)
# g_tot_precision: 95.0661853188929 (%)
# g_tot_recall: 95.0661853188929 (%)
# bb_tot_recall: 95.00601684717208 (%)
# bb_tot_f1: 95.00601684717208 (%)
# total samples covered by g: 1662 (out of 2003)
# total coverage by g: 0.8297553669495756

G Auroc: 0.9599988019857087
BB Auroc: 0.9685371496744146
g_tot_acc: 92.29074889867842 (%)
bb_tot_acc: 92.84140969162996 (%)
g_tot_f1: 92.29074889867842 (%)
g_tot_precision: 92.29074889867842 (%)
g_tot_recall: 92.29074889867842 (%)
bb_tot_recall: 92.84140969162996 (%)
bb_tot_f1: 92.84140969162996 (%)
total samples covered by g: 1816 (out of 2003)
total coverage by g: 0.9066400399400899


<IPython.core.display.Javascript object>

In [14]:
0.9565059403437817 * 0.8637044433349975, 90.11 * 0.8637044433349975

(0.8261384307512443, 77.82840738891663)

<IPython.core.display.Javascript object>

In [17]:
# Final AUROC (Symbolic + Residual) and Accuracy
0.8261384307512443 + 0.10049071392910633, 77.82840738891663 + 10.190818771842235

(0.9266291446803506, 88.01922616075886)

<IPython.core.display.Javascript object>

In [10]:
1 - 0.8637044433349975

0.13629555666500248

<IPython.core.display.Javascript object>

In [79]:
# AUROC of residual
0.7373 * 0.13629555666500248, 74.77 * 0.13629555666500248

(0.10049071392910633, 10.190818771842235)

<IPython.core.display.Javascript object>

In [97]:
x = [
    val_tensor_preds_1.size(0) / 2003,
    val_tensor_preds_2.size(0) / 2003,
    val_tensor_preds_3.size(0) / 2003,
    val_tensor_preds_4.size(0) / 2003,
    val_tensor_preds_5.size(0) / 2003,
    0.13629555666500248
]

print(x, sum(x), 1 - sum(x))
# x = [
#     99.9 * (0.391), 
#     97 * (0.25), 
#     84 * (0.1203), 
#     85 * (0.10), 
#     83 * (0.060)
# ]

print(sum(x))




[0.40788816774837744, 0.08936595107338992, 0.14777833250124814, 0.08337493759360959, 0.13529705441837245, 0.13629555666500248] 1.0 0.0
1.0


<IPython.core.display.Javascript object>

In [83]:
(
    torch.sum(val_tensor_preds_1.argmax(dim=1) == val_tensor_y_1)
    / val_tensor_preds_1.size(0)
).item() 

0.9987760186195374

<IPython.core.display.Javascript object>

In [80]:
torch.sum(val_tensor_preds_1.argmax(dim=1) == val_tensor_y_1) / val_tensor_preds_1.size(
    0
) * x[0], torch.sum(
    val_tensor_preds_2.argmax(dim=1) == val_tensor_y_2
) / val_tensor_preds_2.size(
    0
) * x[
    1
], torch.sum(
    val_tensor_preds_3.argmax(dim=1) == val_tensor_y_3
) / val_tensor_preds_3.size(
    0
) * x[
    2
], torch.sum(
    val_tensor_preds_4.argmax(dim=1) == val_tensor_y_4
) / val_tensor_preds_4.size(
    0
) * x[
    3
], torch.sum(
    val_tensor_preds_5.argmax(dim=1) == val_tensor_y_5
) / val_tensor_preds_5.size(
    0
) * x[
    4
], 10.190818771842235

(tensor(0.4074),
 tensor(0.0869),
 tensor(0.1143),
 tensor(0.0709),
 tensor(0.0989),
 10.190818771842235)

<IPython.core.display.Javascript object>

In [66]:
torch.sum(val_tensor_preds_1.argmax(dim=1) == val_tensor_y_1) / val_tensor_preds_1.size(
    0
), torch.sum(
    val_tensor_preds_2.argmax(dim=1) == val_tensor_y_2
) / val_tensor_preds_2.size(
    0
), torch.sum(
    val_tensor_preds_3.argmax(dim=1) == val_tensor_y_3
) / val_tensor_preds_3.size(
    0
), torch.sum(
    val_tensor_preds_4.argmax(dim=1) == val_tensor_y_4
) / val_tensor_preds_4.size(
    0
), torch.sum(
    val_tensor_preds_5.argmax(dim=1) == val_tensor_y_5
) / val_tensor_preds_5.size(
    0
)

(tensor(0.9988),
 tensor(0.9721),
 tensor(0.7736),
 tensor(0.8503),
 tensor(0.7306))

<IPython.core.display.Javascript object>

In [88]:
p = [0.9988 * x[0], 0.9721 * x[1], 0.7736 * x[2] , 0.8503* x[3], 0.7306 * x[4]]
sum(p), p


(0.7783343984023965,
 [0.4073987019470794,
  0.08687264103844233,
  0.11432131802296555,
  0.07089370943584623,
  0.09884802795806291])

<IPython.core.display.Javascript object>

In [89]:
0.13629555666500248

0.13629555666500248

<IPython.core.display.Javascript object>

In [93]:
sum(p) * 100 + 10.190818771842235

88.02425861208188

<IPython.core.display.Javascript object>

In [99]:
auroc = [
    0.9093137254901961 * x[0],
    0.8474576271186441 * x[1],
    0.8230006720430108 * x[2],
    0.8502415458937198 * x[3],
    0.7599325337331333 * x[4],
    0.7373 * x[5],
]

auroc, sum(auroc)

([0.37089830939864715,
  0.07573385684185586,
  0.12162166696192271,
  0.07088883582838303,
  0.1028166333707834,
  0.10049071392910633],
 0.8424500163306985)

<IPython.core.display.Javascript object>

In [104]:
a1 = [
    0.37089830939864715,
    0.11573385684185586,
    0.16162166696192271,
    0.07488883582838303,
    0.1028166333707834,
    0.10049071392910633,
]
sum(a1)

0.9264500163306985

<IPython.core.display.Javascript object>

In [57]:
path = "/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/HAM10k/BB/Inception_V3"
gt = torch.tensor(np.load(os.path.join(path, "out_put_GT_prune.npy")))
pred_bb = torch.load(os.path.join(path, "out_put_predict_bb.pt"))

<IPython.core.display.Javascript object>

In [58]:
idx = (gt == 1).nonzero(as_tuple=True)[0]
print(idx.size())
print(torch.sum(pred_bb[idx]))

torch.Size([326])
tensor(294)


<IPython.core.display.Javascript object>

In [56]:
gt_g = torch.tensor(gt_tot_np)
idx_g = (gt_g == 1).nonzero(as_tuple=True)[0]
print(torch.sum(gt_g == 1))
print(torch.sum(torch.tensor(preds_tot_np[idx_g])))

tensor(257)
tensor(216)


<IPython.core.display.Javascript object>